In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import train_test_split

from scipy import sparse
from IPython.display import Image

from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing

sns.set(style="ticks", color_codes=True)
%matplotlib inline

import os
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from datetime import datetime
import dateutil

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

#For re-sizing
from skimage.transform import resize
from scipy import misc
import matplotlib.pyplot as plt
import skimage

import mdptoolbox.example
import cvlib as cv
from matplotlib.pyplot import imshow
from PIL import Image, ImageStat
import cv2
from cvlib.object_detection import draw_bbox

import math

In [88]:
def image_colorfulness(image):
    # split the image into its respective RGB components
    (B, G, R) = cv2.split(image.astype("float"))
 
    # compute rg = R - G
    rg = np.absolute(R - G)
 
    # compute yb = 0.5 * (R + G) - B
    yb = np.absolute(0.5 * (R + G) - B)
 
    # compute the mean and standard deviation of both `rg` and `yb`
    (rbMean, rbStd) = (np.mean(rg), np.std(rg))
    (ybMean, ybStd) = (np.mean(yb), np.std(yb))
 
    # combine the mean and standard deviations
    stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
    meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
 
    # derive the "colorfulness" metric and return it
    return stdRoot + (0.3 * meanRoot)

def percieved_brightness(im_file):
   im = Image.open(im_file)
   stat = ImageStat.Stat(im)
   r,g,b = stat.mean
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

def brightness(im_file):
   im = Image.open(im_file).convert('L')
   stat = ImageStat.Stat(im)
   return stat.mean[0]

def detect_genders(face, conf, img_filename):
    # apply face detection
    img = cv2.imread(img_filename)
    #face, conf = cv.detect_face(img)
    gender_array = []
    # loop through detected faces
    for f in face:

        (startX,startY) = f[0],f[1]
        (endX,endY) = f[2],f[3]

        # draw rectangle over face
        cv2.rectangle(img, (startX,startY), (endX,endY), (0,255,0), 2)

        face_crop = np.copy(img[startY:endY, startX:endX])

        # apply gender detection
        (label, confidence) = cv.detect_gender(face_crop)

        #print(confidence)
        #print(label)

        idx = np.argmax(confidence)
        label = label[idx]
        gender_array.append(label)
    return gender_array


In [89]:
paths = ["alexachung", "bucketlistjourney", "mariekondo", "jamieoliver", "gypsea_lust", 
         "kimkardashian", "tombrady", "ocasio2018"]
# filelist = os.listdir(path)
followers = {}
followers["alexachung"] = float(3300000)
followers["bucketlistjourney"] = float(100000)
followers["mariekondo"] = float(3000000)
followers["jamieoliver"] = float(6700000)
followers["gypsea_lust"] = float(2100000)
followers["kimkardashian"] = float(138000000)
followers["tombrady"] = float(6100000)
followers["ocasio2018"] = float(3400000)

df = pd.DataFrame(columns=['filename', 'followers', 'num_faces', 'percieved_brightness', 
                           'people', 'brightness', 'avg_r', 'avg_g', 'avg_b', 'likes_norm', 'num_female', 'num_male', 'likes' ])


In [90]:
image_arrays = []
for p in paths:
    filelist = os.listdir(p)
    for file in filelist:
        if str(file)[-1] != "g":
            continue
        file_name = p + str("/") + str(file)
        pic = load_img(file_name)
        arr = img_to_array(pic)
        image_arrays.append((p, str(file), arr))
        df = df.append({'filename' : file_name} , ignore_index=True)

In [100]:
dataDict = {}
for path in paths:
    with open(path + str("/") + path + str(".json")) as f:
        data = json.load(f)
    dataDict[path] = data

likes = {}
final_arr = []
likes_array = []
for img in image_arrays:
    jsonData = dataDict[img[0]]
    for i in range(len(jsonData['GraphImages'])):
        if img[1] in jsonData['GraphImages'][i]["display_url"]:
            
            image_file_name = img[0] + str("/") + img[1] 
            
            image = Image.open(image_file_name, 'r')
            image_array = img_to_array(image)
            image_cv2 = cv2.imread(image_file_name)

            # Add likes to the df
            likes[img[1]] = jsonData['GraphImages'][i]["edge_media_preview_like"]["count"]
            df.loc[df['filename'] == image_file_name,'likes_norm'] = jsonData['GraphImages'][i]["edge_media_preview_like"]["count"]/float(followers[img[0]])
            df.loc[df['filename'] == image_file_name,'likes'] = jsonData['GraphImages'][i]["edge_media_preview_like"]["count"]

            # Add number of faces 
            faces, confidences = cv.detect_face(image_cv2)
            df.loc[df['filename'] == image_file_name, 'num_faces'] = len(confidences)

            # Add objects
            bbox, label, conf = cv.detect_common_objects(image_cv2)
            #df.loc[df['filename'] == image_file_name, 'objects'] = pd.Series(label)
            
            # Add people
            df.loc[df['filename'] == image_file_name, 'people'] = label.count('person')
            
            # Add color averages
            means = cv2.mean(image_array)
            df.loc[df['filename'] == image_file_name, 'avg_r'] = int(means[0])
            df.loc[df['filename'] == image_file_name, 'avg_g'] = int(means[1])
            df.loc[df['filename'] == image_file_name, 'avg_b'] = int(means[2])
            
            
            #Add "colorfulness" metric
            #df.loc[df['filename'] == image_file_name,'colorfulness'] = image_colorfulness(image_array)
            
            # Add brightness and percieved brightness
            df.loc[df['filename'] == image_file_name, 'brightness'] = brightness(image_file_name)
            df.loc[df['filename'] == image_file_name, 'percieved_brightness'] = percieved_brightness(image_file_name)
            
            
            # Add followers
            df.loc[df['filename'] == image_file_name, 'followers'] = followers[img[0]]
            
            #Extract gender ratio of photo. 
            gender_arary = detect_genders(faces, confidences, image_file_name)
            
            if(len(faces) > 0):
                df.loc[df['filename'] == image_file_name, 'num_male'] = gender_arary.count('man')
                df.loc[df['filename'] == image_file_name, 'num_female'] = gender_arary.count('woman')
            else:
                df.loc[df['filename'] == image_file_name, 'num_male'] = 0
                df.loc[df['filename'] == image_file_name, 'num_female'] = 0
                            
df = df.dropna()


In [102]:
# Normalize numerical cols in DF
df['percieved_brightness'] = (df['percieved_brightness'] - df['percieved_brightness'].mean())
df['brightness'] = (df['brightness'] - df['brightness'].mean())
df['avg_r'] = (df['avg_r'] - df['avg_r'].mean())
df['avg_g'] = (df['avg_g'] - df['avg_g'].mean())
df['avg_b'] = (df['avg_b'] - df['avg_b'].mean())
print(df)

                                              filename followers num_faces  \
0    alexachung/57939604_1405275606280969_227927799...   3.3e+06         1   
1    alexachung/57487966_302650897331937_5403985171...   3.3e+06         0   
3    alexachung/58409673_1888377867934548_313100938...   3.3e+06         0   
4    alexachung/57511927_446139556140698_1423052042...   3.3e+06         1   
5    alexachung/58994690_801384873562945_4583796651...   3.3e+06         0   
6    alexachung/57568586_144722209994647_2830766459...   3.3e+06         0   
7    alexachung/58453539_318563102171701_5107544445...   3.3e+06         1   
8    alexachung/57598467_109244776838216_8820137001...   3.3e+06         1   
9    alexachung/59444228_294271814828332_6767495519...   3.3e+06         0   
10   bucketlistjourney/59767226_464352457440775_604...    100000         0   
11   bucketlistjourney/57303614_417134662419576_839...    100000         0   
12   bucketlistjourney/56551834_425069051389781_109...    100000

In [82]:
category = pd.cut(df.percieved_brightness,[-150, -60, -40, -0, 20, 40, 60, 150])
category = category.to_frame()
category.columns = ['percieved_brightness_discrete']
df_new = pd.concat([df,category],axis = 1)

category = pd.cut(df.brightness,[-150, -60, -40, -0, 20, 40, 60, 150])
category = category.to_frame()
category.columns = ['brightness_discrete']
df_new = pd.concat([df_new, category],axis = 1)

category = pd.cut(df.avg_r,[-150, -60, -40, -0, 20, 40, 60, 150])
category = category.to_frame()
category.columns = ['avg_r_disc']
df_new = pd.concat([df_new, category],axis = 1)

category = pd.cut(df.avg_g,[-150, -60, -40, -0, 20, 40, 60, 150])
category = category.to_frame()
category.columns = ['avg_g_disc']
df_new = pd.concat([df_new, category],axis = 1)

category = pd.cut(df.avg_b,[-150, -60, -40, -0, 20, 40, 60, 150])
category = category.to_frame()
category.columns = ['avg_b_disc']
df_new = pd.concat([df_new, category],axis = 1)

#concatenate age and its bin
print(df_new)
print()

                                              filename    followers  \
0    alexachung/57939604_1405275606280969_227927799...    3300000.0   
1    alexachung/57487966_302650897331937_5403985171...    3300000.0   
3    alexachung/58409673_1888377867934548_313100938...    3300000.0   
4    alexachung/57511927_446139556140698_1423052042...    3300000.0   
5    alexachung/58994690_801384873562945_4583796651...    3300000.0   
6    alexachung/57568586_144722209994647_2830766459...    3300000.0   
7    alexachung/58453539_318563102171701_5107544445...    3300000.0   
8    alexachung/57598467_109244776838216_8820137001...    3300000.0   
9    alexachung/59444228_294271814828332_6767495519...    3300000.0   
10   bucketlistjourney/59767226_464352457440775_604...     100000.0   
11   bucketlistjourney/57303614_417134662419576_839...     100000.0   
12   bucketlistjourney/56551834_425069051389781_109...     100000.0   
13   bucketlistjourney/59121268_335244693724167_156...     100000.0   
14   b

In [129]:
# LOG REGRESSION

train, test = train_test_split(df, test_size=0.2)

X_train = train[['num_faces', 'percieved_brightness', 'brightness', 'people', 'avg_r', 'avg_g', 'avg_b', 'num_female', 'num_male',]]
y_train = np.ravel(train[['likes_norm']])

X_test = test[['num_faces', 'percieved_brightness', 'brightness', 'people', 'avg_r', 'avg_g', 'avg_b',  'num_female', 'num_male',]]
y_test = np.ravel(test[['likes_norm']])

print(y_test)

y_train = np.multiply(y_train, 100)
y_train = y_train.astype(int)

y_test = np.multiply(y_test, 100)
y_test = y_test.astype(int)

           
print(y_train)
#X_train, X_test, y_train, y_test = train_test_split(img_values, new_y, test_size=0.1, random_state=0)

clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=2000,
                         class_weight='balanced', verbose=5).fit(X_train, y_train)

y = clf.predict(X_test)

print(mean_squared_error(y_test, y))
print(y_test)
print(np.ravel(test['filename']))
print(test)
print(y)

[0.027783809523809523 0.01537 0.09238491803278688 0.026615714285714284
 0.06845323529411765 0.0033015151515151516 0.05183901639344262 0.01268
 0.1406214705882353 0.06452617647058824 0.02532576811594203
 0.09412882352941176 0.007262727272727273 0.01516 0.018406
 0.05351655737704918 0.06253590163934426]
[ 2  4 21  1  1  8  1  2  0  0  0 24  9  0  1  0  0  0  2  7  6  0  1  2
  0  1  2  2  8  2  3  3  1  3  1  1  1  1  0  2  7  0  0  6  3  2  1  1
  2  0  1  8  1  1  1  0  5  2  0 11  3  1  1  2]
17.294117647058822
[ 2  1  9  2  6  0  5  1 14  6  2  9  0  1  1  5  6]
['gypsea_lust/57012713_342228239741992_3191860124595154034_n.jpg'
 'bucketlistjourney/56723983_2406294326067674_5616514260652929922_n.jpg'
 'tombrady/56770808_399294940654325_6593980193733087902_n.jpg'
 'gypsea_lust/59423327_443314916240152_3266229492340529069_n.jpg'
 'ocasio2018/50691545_346000186004538_2945915006050350194_n.jpg'
 'alexachung/57568586_144722209994647_2830766459764292268_n.jpg'
 'tombrady/57506533_29466057144

/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:719: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


In [130]:
print(test)
print(df_new.loc[113])

                                              filename followers num_faces  \
49   gypsea_lust/57012713_342228239741992_319186012...   2.1e+06         0   
15   bucketlistjourney/56723983_2406294326067674_56...    100000         0   
117  tombrady/56770808_399294940654325_659398019373...   6.1e+06         1   
47   gypsea_lust/59423327_443314916240152_326622949...   2.1e+06         0   
140  ocasio2018/50691545_346000186004538_2945915006...   3.4e+06         2   
6    alexachung/57568586_144722209994647_2830766459...   3.3e+06         0   
122  tombrady/57506533_294660571448999_657438808910...   6.1e+06         0   
13   bucketlistjourney/59121268_335244693724167_156...    100000         0   
134  ocasio2018/51759811_406233833477172_4333460932...   3.4e+06         0   
132  ocasio2018/52572824_2048897855225268_566729145...   3.4e+06         1   
96   kimkardashian/57840131_644723422642924_4839165...  1.38e+08         1   
137  ocasio2018/52736729_263180281279482_6627642289...   3.4e+06

In [ ]:
gypsea_lust/55914433_2636214979727755_3426739042950481133_n.jpg

In [ ]:
gypsea_lust/55914433_2636214979727755_3426739042950481133_n.jpg

In [ ]:
Q[observation] 

In [112]:
def Q():
    


def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    
    Args:
        Q: A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA (see below)
        epsilon: The probability to select a random action. Float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [ ]:
def q_learning(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """
    Q-Learning algorithm: Off-policy TD control. Finds the optimal greedy policy
    while following an epsilon-greedy policy
    
    Args:
        env: OpenAI environment.
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        alpha: TD learning rate.
        epsilon: Chance to sample a random action. Float between 0 and 1.
    
    Returns:
        A tuple (Q, episode_lengths).
        Q is the optimal action-value function, a dictionary mapping state -> action values.
        stats is an EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    for i_episode in range(num_episodes):
        # Print out which episode we're on, useful for debugging.
        if (i_episode + 1) % 100 == 0:
            print("\rEpisode {}/{}.".format(i_episode + 1, num_episodes), end="")
            sys.stdout.flush()
        
        # Reset the environment and pick the first action
        state = env.reset()
        
        # One step in the environment
        # total_reward = 0.0
        for t in itertools.count():
            
            # Take a stepgypsea_lust/55914433_2636214979727755_3426739042950481133_n.jpg
            action_probs = policy(state) (THIS IS A
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _ = env.step(action)

            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t
            
            # TD Update
            best_next_action = np.argmax(Q[next_state])    
            td_target = reward + discount_factor * Q[next_state][best_next_action]
            td_delta = td_target - Q[state][action]
            Q[state][action] += alpha * td_delta
                
            if done:
                break
                
            state = next_state
    
    return Q, stats